# P3 

In [1]:
import pandas as pd 
from bs4 import BeautifulSoup
import requests
import re

## Question 1

(5 points) Scrape  https://www.data.gov to identify and print out the number of data sets available.

In [2]:
html = "https://www.data.gov"

In [3]:
soup = BeautifulSoup(requests.get(html).text, 'html5lib')

In [4]:
datasets = soup.find(class_="text-center getstarted")
df = datasets.find_all('a')
for i in df:
    print("The number of data sets available = ",i.text)

The number of data sets available =  250,019 datasets


## Question 2 

(20 points) Identify the number of U.S. Senate votes in the 116th Congress that were rejected 
by a margin of less than 5 votes. Also, provide a list of which votes (#) match this criteria.
Data available at: https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_116_1.htm

In [5]:
import lxml.html as lh

In [6]:
url='https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_116_1.htm'

In [7]:
page = requests.get(url)
doc = lh.fromstring(page.content)

In [8]:
tr_elements = doc.xpath('//tr')

In [9]:
tr_elements = doc.xpath('//tr')

col=[] #Empty list
i=0

for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    #print('%d:"%s"'%(i,name))
    col.append((name,[])) #Storing the header in the list

In [10]:
for j in range(1,len(tr_elements)):
    
    T=tr_elements[j]
    
    if len(T)!=5:                   #checking the size of row so that we don't extract data from other table
        break
    
    i=0
    
    for t in T.iterchildren():     #Iterate through each element of the row
        data=t.text_content() 
        
        if i>0:                    #Check if row is empty
            try:                   #Convert any numerical value to integers
                data=int(data)
            except:
                pass
        
        col[i][1].append(data)     #Append the data
        i+=1

In [11]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [12]:
df2 = df[df['Result'].isin(['Rejected'])]

In [13]:
val = df2['Vote (Tally)']

In [14]:
vote_difference=[]
for i in range(0,len(val)):
    str1=val.iloc[i].split('(')[2]
    str1=str1[:-1]
    
    str2=str1.split('-')
    
    p1=int(str2[0])
    p2=int(str2[1])
    vote_difference.append(abs(p1-p2))

In [15]:
lis2=[]
for i in range(0,len(vote_difference)):
    if(vote_difference[i]<5):
        lis2.append(i)

In [23]:
lis2 #dataframe index of rejection by a margin of less than 5 votes.

[2, 14, 15, 20, 21]

In [16]:
df2.iloc[[2, 14, 15, 20, 21],]

,Vote (Tally),Result,Question: Description,Issue,Date
20,(00305) 305 (47-48),Rejected,On the Resolution: S. Res. 333; A resolution i...,S.Res. 333,Sep 25
268,(00057) 57 (51-48),Rejected,On Cloture on the Motion to Proceed: Motion to...,S.Res. 50,Apr 02
269,(00056) 56 (46-48),Rejected,On the Cloture Motion: Motion to Invoke Clotur...,H.R. 268,Apr 01
316,(00009) 9 (50-47),Rejected,On the Cloture Motion S.Amdt. 5: Motion Invoke...,H.R. 268,Jan 24
318,(00007) 7 (48-47),Rejected,On Cloture on the Motion to Proceed: Motion to...,S. 109,Jan 17


## Question 3 

(20 points) Report the total number of female babies whose names start with 'K' so far this decade. Data available at: https://www.ssa.gov/OACT/babynames/decades/names2010s.html

In [79]:
url='https://www.ssa.gov/OACT/babynames/decades/names2010s.html'

In [80]:
page = requests.get(url)
doc = lh.fromstring(page.content)

In [81]:
tr_elements = doc.xpath('//tr')

In [82]:
tr_elements = doc.xpath('//tr')

col=[] #Empty list
i=0

for t in tr_elements[1]:
    i+=1
    name=t.text_content()
    #print('%d:"%s"'%(i,name))
    col.append((name,[])) #Storing the header in the list

In [83]:
col[1] = ('Name_boy',[])
col[3] = ('Name_girl',[])

In [91]:
for j in range(2,len(tr_elements)):
    
    T=tr_elements[j]
    
    if len(T)!=5:                   #checking the size of row so that we don't extract data from other table
        break
    
    i=0
    
    for t in T.iterchildren():     #Iterate through each element of the row
        data=t.text_content() 
        
        if i>0:                    #Check if row is empty
            try:                   #Convert any numerical value to integers
                data=int(data)
            except:
                pass
        
        col[i][1].append(data)     #Append the data
        i+=1

In [92]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [105]:
df = df[['Name_girl','Number']]

In [108]:
df2=df[df['Name_girl'].str.match('K')]

In [115]:
df2

,Name_girl,Number
45,Kaylee,"41,958"
70,Kennedy,"34,828"
73,Kylie,"33,775"
81,Khloe,"31,710"
82,Katherine,"31,635"
92,Kayla,"28,606"
108,Kimberly,"25,688"
127,Kinsley,"22,722"
148,Kendall,"19,005"
164,Kaitlyn,"17,623"


In [118]:
print("Total number of female babies whose names start with 'K' so far this decade:")
df2['Number'].sum(axis = 0)

Total number of female babies whose names start with 'K' so far this decade:


'41,95834,82833,77531,71031,63528,60625,68822,72219,00517,62315,911'

## Question 4 

(30 points) Use Python to scrape the article posted to TechCrunch.com and present the articles on the front page in a table with the following pieces of information:

* article title
* publish date
* author

Note: be respectful to the website (obey robots.txt information) and do not repeatedly access the site.

In [160]:
url = 'https://techcrunch.com/'

In [163]:
source = requests.get(url).text
soup = BeautifulSoup(source,'lxml')

In [200]:
df = pd.DataFrame(columns=['Article Title', 'Publish Date', 'Author'])
for article in soup.find_all('div',class_='post-block post-block--image post-block--unread'):
    df.loc[len(df)] = [article.a.text.split('\t')[4], article.time.text.split('\t')[2], article.span.a.text.split('\t')[4]]

In [201]:
df

,Article Title,Publish Date,Author
0,Apple’s China stance makes for strange politic...,"Oct 18, 2019",Jonathan Shieber
1,AI is helping scholars restore ancient Greek t...,"Oct 18, 2019",Devin Coldewey
2,Adam Neumann planned for his children and gran...,"Oct 18, 2019",Connie Loizos
3,VR/AR startup valuations reach $45 billion (on...,"Oct 18, 2019",Tim Merel
4,Tilting Point acquires game monetization start...,"Oct 18, 2019",Anthony Ha
5,Where are US fintech’s next billion-dollar sta...,"Oct 18, 2019",Jonathan Shieber
6,Alphabet’s Wing begins making first commercial...,"Oct 18, 2019",Darrell Etherington
7,Daily Crunch: Zuckerberg has thoughts on free ...,"Oct 18, 2019",Anthony Ha
8,Harley-Davidson has resumed production of the ...,"Oct 18, 2019",Kirsten Korosec
9,Who will own the future of transportation?,"Oct 18, 2019",Kirsten Korosec
